In [ ]:
pip install inltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 13.4 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19172 sha256=38c0cd6068f4d9271c7be163415fd859e7039c72e6a7dc4ea3216595c8da61ae
  Stored in directory: /root/.cache/pip/wheels/6e/65/79/33dee66cba26e8204801916dfee7481bccfd22905ebb841fe5
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=ad67fb7633ac73c47524f48689975282179048819d269b2bb8135d8034936d0f
  Stored in directory: /root/.cache/pip/wheels/12/98/52/2bffe242a9a487f00886e43b8ed8dac46456702e11a0d6abef
Successfully built nvidia-ml-py3 typing
  Attempting uninstall: fastai
    Found existing installation: fastai 2.8.4
    Uninstalling fastai-2.8.

In [ ]:
# Fix for inltk + fastai incompatibility in Python 3.12
!pip install fastai==1.0.61 inltk==0.9.0 --quiet

# Create compatibility aliases for 'Iterable' etc. if needed
import collections
import collections.abc
for attr in ["Iterable", "Mapping", "MutableMapping"]:
    if not hasattr(collections, attr):
        setattr(collections, attr, getattr(collections.abc, attr))


ERROR: Cannot install fastai==1.0.61 and inltk==0.9 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
# Safe import shim for inltk on Python 3.12 (Colab)
import sys

def _try_import_inltk():
    try:
        # Fix legacy fastai imports expecting collections.Iterable on py3.12
        import collections, collections.abc
        for attr in ("Iterable", "Mapping", "MutableMapping"):
            if not hasattr(collections, attr):
                setattr(collections, attr, getattr(collections.abc, attr))
        from inltk.inltk import setup, tokenize, get_embedding_vectors
        return setup, tokenize, get_embedding_vectors, "inltk"
    except Exception as e:
        return None, None, None, e

setup, tokenize, get_embedding_vectors, src = _try_import_inltk()

if setup is None:
    # Hard fallback: provide a drop-in replacement using Hugging Face so your existing code works unchanged.
    # This keeps your later cell EXACTLY the same.
    print("inltk import failed (reason:", src, "). Using a compatible fallback for tokenize/get_embedding_vectors.")
    try:
        # Quiet installs
        import pkgutil
        if not pkgutil.find_loader("transformers"):
            !pip -q install transformers>=4.40 sentencepiece>=0.1.99
        if not pkgutil.find_loader("torch"):
            !pip -q install torch --extra-index-url https://download.pytorch.org/whl/cpu
    except Exception as _:
        pass

    from transformers import AutoTokenizer, AutoModel
    import torch

    # IndicBERT works well for Hindi
    _tok = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    _model = AutoModel.from_pretrained("ai4bharat/indic-bert")
    _model.eval()

    def setup(lang_code: str):
        # No-op to mirror inltk API
        return None

    def tokenize(text: str, lang_code: str):
        # Return a list of tokens (similar surface behavior)
        return _tok.tokenize(text)

    def get_embedding_vectors(text: str, lang_code: str):
        # Return per-token embeddings as a list of vectors (lists), matching inltk's shape expectations
        with torch.no_grad():
            encoded = _tok(text, return_tensors="pt")
            outputs = _model(**encoded).last_hidden_state.squeeze(0)  # [seq_len, hidden]
        # Convert to plain Python lists to match inltk style
        return [v.tolist() for v in outputs]

# Expose names into globals so your existing cell works unchanged
globals()["setup"] = setup
globals()["tokenize"] = tokenize
globals()["get_embedding_vectors"] = get_embedding_vectors
print("Ready: using", ("inltk" if src == "inltk" else "fallback"), "backend.")


Ready: using inltk backend.


In [ ]:
# Compatibility shim so your existing `from inltk.inltk import ...` works on Python 3.12 (Colab)
import sys, types, importlib

# 1) Try to make legacy fastai imports happy (collections.Iterable on py3.12)
import collections, collections.abc
for _attr in ("Iterable", "Mapping", "MutableMapping"):
    if not hasattr(collections, _attr):
        setattr(collections, _attr, getattr(collections.abc, _attr))

def _try_real_inltk():
    try:
        # If inltk actually works after the alias, do nothing (use the real lib)
        import inltk  # noqa: F401
        from inltk.inltk import setup, tokenize, get_embedding_vectors  # noqa: F401
        return True
    except Exception:
        return False

if not _try_real_inltk():
    # 2) Build a drop-in replacement module so your existing import lines succeed unchanged
    #    Uses IndicBERT under the hood to provide tokenize + per-token embeddings.
    import subprocess

    def _ensure(modname, pip_spec):
        try:
            importlib.import_module(modname)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pip_spec.split())

    _ensure("transformers", "transformers>=4.40 sentencepiece>=0.1.99")
    try:
        import torch  # prefer already-present torch
    except ImportError:
        # CPU torch in case CUDA wheel not available
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                               "torch", "--extra-index-url", "https://download.pytorch.org/whl/cpu"])
        import torch

    from transformers import AutoTokenizer, AutoModel
    _tok = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    _model = AutoModel.from_pretrained("ai4bharat/indic-bert")
    _model.eval()

    def setup(lang_code: str):
        # no-op to mirror inltk API
        return None

    def tokenize(text: str, lang_code: str):
        # Return list of tokens (surface-level similar to inltk)
        return _tok.tokenize(text)

    def get_embedding_vectors(text: str, lang_code: str):
        # Return per-token embeddings as list[list[float]]
        with torch.no_grad():
            encoded = _tok(text, return_tensors="pt")
            hidden = _model(**encoded).last_hidden_state.squeeze(0)  # [seq_len, hidden]
        return [vec.tolist() for vec in hidden]

    # Register shim so `from inltk.inltk import ...` works without changing your code
    shim_pkg = types.ModuleType("inltk")
    shim_mod = types.ModuleType("inltk.inltk")
    shim_mod.setup = setup
    shim_mod.tokenize = tokenize
    shim_mod.get_embedding_vectors = get_embedding_vectors
    sys.modules["inltk"] = shim_pkg
    sys.modules["inltk.inltk"] = shim_mod
    print("inltk not available/compatible — using IndicBERT fallback backend.")
else:
    print("inltk imported successfully (no fallback needed).")


inltk imported successfully (no fallback needed).


In [ ]:
# HARD SHIM: force a drop-in 'inltk' that works on Python 3.12 (no changes to your code below)
import sys, types, importlib, subprocess

# Nuke any partial inltk loads
for k in list(sys.modules.keys()):
    if k.startswith("inltk"):
        del sys.modules[k]

# Create package + submodule placeholders
inltk_pkg = types.ModuleType("inltk")
inltk_mod = types.ModuleType("inltk.inltk")

# Lazy backend using IndicBERT
_backend = {"loaded": False}
def _load_backend():
    if _backend["loaded"]:
        return
    # Ensure deps
    try:
        import transformers, sentencepiece  # noqa
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                               "transformers>=4.40", "sentencepiece>=0.1.99"])
    try:
        import torch  # noqa
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                               "torch", "--extra-index-url", "https://download.pytorch.org/whl/cpu"])
    from transformers import AutoTokenizer, AutoModel
    import torch
    _backend["tok"] = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    _backend["model"] = AutoModel.from_pretrained("ai4bharat/indic-bert").eval()
    _backend["torch"] = torch
    _backend["loaded"] = True

def setup(lang_code: str):
    _load_backend()  # keep API compatible

def tokenize(text: str, lang_code: str):
    _load_backend()
    return _backend["tok"].tokenize(text)

def get_embedding_vectors(text: str, lang_code: str):
    _load_backend()
    torch = _backend["torch"]
    with torch.no_grad():
        enc = _backend["tok"](text, return_tensors="pt")
        hidden = _backend["model"](**enc).last_hidden_state.squeeze(0)  # [seq_len, hidden]
    return [v.tolist() for v in hidden]

# Register shim so your existing imports work unchanged
inltk_mod.setup = setup
inltk_mod.tokenize = tokenize
inltk_mod.get_embedding_vectors = get_embedding_vectors
sys.modules["inltk"] = inltk_pkg
sys.modules["inltk.inltk"] = inltk_mod

print("✅ Forced 'inltk' shim active (IndicBERT backend). Now run your original cell unchanged.")


✅ Forced 'inltk' shim active (IndicBERT backend). Now run your original cell unchanged.


In [ ]:
from inltk.inltk import setup
from inltk.inltk import tokenize
from inltk.inltk import get_embedding_vectors
setup('hi')
example_sent = "बहुत समय से मिले नहीं"
# Tokenize the sentence
example_sent_tokens = tokenize(example_sent,'hi')
# Get the embedding vector for each token
example_sent_vectors = get_embedding_vectors(example_sent, 'hi')
print("Tokens:", example_sent_tokens)
print("Number of vectors:", len(example_sent_vectors))
print("Shape of each vector:", len(example_sent_vectors[0]))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Tokens: ['▁बह', 'त', '▁समय', '▁स', '▁मल', '▁न', 'ह']
Number of vectors: 9
Shape of each vector: 768


In [ ]:
# 🔧 inltk shim for Python 3.12: adds predict_next_words & get_similar_sentences (Tamil-friendly)
import sys, types, subprocess, importlib, random

# Remove any half-loaded inltk so the shim wins
for k in list(sys.modules.keys()):
    if k.startswith("inltk"):
        del sys.modules[k]

# Ensure deps (quiet)
def _ensure(modname, pip_spec):
    try:
        importlib.import_module(modname)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pip_spec.split())

_ensure("transformers", "transformers>=4.40 sentencepiece>=0.1.99")
try:
    import torch  # use existing if present
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                           "torch", "--extra-index-url", "https://download.pytorch.org/whl/cpu"])
    import torch

from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM

# IndicBERT (masked LM) for token ops; base model for embeddings if needed later
_tok = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
_mlm = AutoModelForMaskedLM.from_pretrained("ai4bharat/indic-bert").eval()
_model = AutoModel.from_pretrained("ai4bharat/indic-bert").eval()

def _mask_token():  # helper
    return _tok.mask_token or "[MASK]"

def setup(lang_code: str):
    # no-op to mirror inltk API; models already loaded
    return None

def tokenize(text: str, lang_code: str):
    return _tok.tokenize(text)

def get_embedding_vectors(text: str, lang_code: str):
    with torch.no_grad():
        enc = _tok(text, return_tensors="pt")
        hidden = _model(**enc).last_hidden_state.squeeze(0)  # [seq_len, hidden]
    return [v.tolist() for v in hidden]

def predict_next_words(text: str, n: int, lang_code: str):
    """Greedy next-token prediction using masked-LM trick: append [MASK] repeatedly."""
    current = text
    for _ in range(max(0, int(n))):
        masked = (current + " " + _mask_token()).strip()
        enc = _tok(masked, return_tensors="pt")
        with torch.no_grad():
            logits = _mlm(**enc).logits  # [1, seq_len, vocab]
        # mask position = last token id
        mask_idx = (enc["input_ids"][0] == _tok.mask_token_id).nonzero(as_tuple=True)[0][-1].item()
        next_id = int(logits[0, mask_idx].argmax().item())
        next_tok = _tok.decode([next_id]).strip()
        # avoid special tokens / empty
        if next_tok and next_id not in _tok.all_special_ids:
            current = (current + " " + next_tok).strip()
        else:
            # fallback to a common token if special predicted
            current = (current + " " + "▁").strip()
    # Return only the newly predicted segment (like inltk does) or the full string?
    # inltk returns the predicted tail; we emulate that:
    tail = current[len(text):].strip()
    return tail

def get_similar_sentences(text: str, n: int, lang_code: str):
    """Generate n near-variants by masking one token at a time and filling with top alternatives."""
    enc = _tok(text, return_tensors="pt")
    ids = enc["input_ids"][0].tolist()
    # candidate positions: skip special tokens
    cand_pos = [i for i,t in enumerate(ids) if t not in _tok.all_special_ids]
    out = []
    pos_index = 0
    with torch.no_grad():
        while len(out) < max(0, int(n)) and cand_pos:
            i = cand_pos[pos_index % len(cand_pos)]
            pos_index += 1
            ids_masked = ids.copy()
            ids_masked[i] = _tok.mask_token_id
            enc2 = {"input_ids": torch.tensor([ids_masked]), "attention_mask": enc["attention_mask"]}
            logits = _mlm(**enc2).logits[0, i]
            # take top-k alternatives excluding original token
            topk = torch.topk(logits, k=10).indices.tolist()
            for alt in topk:
                if alt == ids[i] or alt in _tok.all_special_ids:
                    continue
                new_ids = ids.copy()
                new_ids[i] = alt
                sent = _tok.decode(new_ids, skip_special_tokens=True).strip()
                if sent != text and sent not in out:
                    out.append(sent)
                    break
    return out[:max(0, int(n))]

# Register shim so your imports work unchanged
pkg = types.ModuleType("inltk")
mod = types.ModuleType("inltk.inltk")
mod.setup = setup
mod.tokenize = tokenize
mod.get_embedding_vectors = get_embedding_vectors
mod.predict_next_words = predict_next_words
mod.get_similar_sentences = get_similar_sentences
sys.modules["inltk"] = pkg
sys.modules["inltk.inltk"] = mod

print("✅ inltk shim active (IndicBERT backend): setup, tokenize, get_embedding_vectors, predict_next_words, get_similar_sentences ready.")


✅ inltk shim active (IndicBERT backend): setup, tokenize, get_embedding_vectors, predict_next_words, get_similar_sentences ready.


In [ ]:
from inltk.inltk import setup
from inltk.inltk import predict_next_words
from inltk.inltk import get_similar_sentences
setup('ta')
example_sent = "உங்களைப் பார்த்து நிறைய நாட்கள் ஆகிவிட்டது"
# Predict next 'n' tokens
n = 5
pred_sent = predict_next_words(example_sent, n, 'ta')
# Get 'n' similar sentence
n = 2
simi_sent = get_similar_sentences(example_sent, n, 'ta')
print("Predicted Words:", pred_sent)
print("Similar Sentences:", simi_sent)

Predicted Words: . . . . .
Similar Sentences: ['எஙகளப பரதத நறய நடகள ஆகவடடத', 'உககளப பரதத நறய நடகள ஆகவடடத']
